# Computing cell mask

In [1]:
from __future__ import division

## Exercises 1 - read images

In [3]:
# import imread from skimage
from skimage.io import imread
from skimage.util import img_as_float

# read fitc, tritc, hoechst, smoothed_image and composite saved previously
# convert the mages to float

fitc = imread(r'fitc.jpeg')
tritc = imread(r'tritc.jpeg')
hoechst = imread(r'hoechst.jpeg')
smoothed_image = imread(r'smoothed_image.jpeg')
composite = imread(r'composite.jpeg')

fitc = img_as_float(fitc)
tritc = img_as_float(tritc)
hoechst = img_as_float(hoechst)
smoothed_image = img_as_float(smoothed_image)
composite = img_as_float(composite)


#### re-calculate the mask of the nuclei seen previously with nuclei_analysis notebook

In [ ]:
import numpy
from skimage.filters import threshold_otsu

nmask = numpy.zeros(smoothed_image.shape, dtype = bool)
nmask[smoothed_image > threshold_otsu(smoothed_image)] = 1
from skimage.morphology import diamond
se = diamond(1)

from skimage.morphology import closing
from skimage.morphology import opening
from scipy.ndimage import binary_fill_holes 

nmask = opening(closing(nmask, se),se)
nmask = binary_fill_holes(nmask)

## Exercises 2 - Adjust Brightness and Contrast

In [ ]:
# Before doing a segmentation of the the cell image, try to increase the brigtness and the contrast of the image. 
# Note: Play with mathematical operator
# operator '+' or '-' for the brigthness
# operator '*' for contrast



# Visualize the result after having created a composite image using plt.imshow and plt.show


In [ ]:
# Improve the brightness of the image by playing with the gamma.
# Use the operator ^ (** in python) to apply a gamma correction


# Visualize the result


## 2d convolution filter

Linear filtering is useful to perform low-pass filtering (to blur images, remove noise...) and high-pass filtering (to detect edges, sharpen images).

## Exercises 3 - Apply a convolution filter

In [ ]:
# Create a cross-shaped structuring element and asign it to a new variable.
# Structuring elements are small binary images that indicate which pixels 
# should be considered as the 'neighborhood' of the central pixel.
# create a matrix with 3 columns and 3 rows as below
# 0 1 0
# 1 2 1
# 0 1 0
# then divide all values by 6 to obtain a float type matrix. Use the function matrix from numpy. 


# import signal from scipy to get the convolve2d function. Execute the 2d convolution filter on the image. 
# Read the documentation to get more details on the parameters of the function.


In [ ]:
# display the result


In [ ]:
# Create a diamond-shaped structuring element and asign it to a new variable.
# Structuring elements are small binary images that indicate which pixels 
# should be considered as the 'neighborhood' of the central pixel.
# You can use skimage.morphology.diamond to create diamond-shaped structuring element


# Use morphology operation like closing to improve image


# segmenting nuclei using watershed

**Background**

The algorithm identifies and separates objects that stand out of the background (zero). It inverts the image and uses water to fill the resulting valleys (pixels with high intensity in the source image) until another object or background is met. The deepest valleys become indexed first, starting from 1.   
An array of seeds contains a few pixels at the center of each cell labeled by a unique ID number and otherwise surrounded by zeros. The expansion algorithm will start from these central pixels and grow outward until all zeros are overwritten by an ID label. In the case of watershed expansion, one can imagine the seeds as the sources from which water pours into the cells and starts filling them up.   
Here, we will use a distance transform and local maxima for seeding.

## Exercises 4 - Seeded watershed

In [ ]:
# Distance transform on thresholded membranes
# Use the function ndi.distance_transform_edt.


In [ ]:
# Visualize the output.


In [ ]:
# Use ndi.filters.maximum_filter to dilate the distance transform.


In [ ]:
# Retrieve the local maxima (the 'peaks') in the distance transform
# Use the function peak_local_max from the module skimage.feature.


In [ ]:
# Visualize the output
# you can first plot the original input (or the smoothed) image and
# then plot the seeds on top of it before showing both with 'plt.show'.


In [ ]:
# Label the seeds
# Use connected component labeling to give each cell seed a unique ID number.


In [ ]:
# Perform watershed
# Use the function watershed from the module skimage.morphology.
# Use the labeled nuclei seeds and the smoothed nuclei image as input.


In [ ]:
# Show the result as transparent overlay over the smoothed input image


# segmenting cells using Voronoi

Find boundaries between adjacent regions in an image, where seeds have been already identified in the individual regions to be segmented. The method finds the Voronoi region of each seed on a manifold with a metric controlled by local image properties. The method is motivated by the problem of finding the borders of cells in microscopy images, given a labelling of the nuclei in the images.

See 
Jones T.R., Carpenter A., Golland P. (2005) Voronoi-Based Segmentation of Cells on Image Manifolds. In: Liu Y., Jiang T., Zhang C. (eds) Computer Vision for Biomedical Image Applications. CVBIA 2005. Lecture Notes in Computer Science, vol 3765. Springer, Berlin, Heidelberg

## Exercises 5 - A modified voronoi algorithm

In [ ]:
# The label function from skimage.measure module finds all connected sets of pixels other than the background, 
# and relabels these sets with a unique increasing integer.


In [ ]:
#  We apply a sobel filter from skimage.filters to detect edges from cell body and serve as input 
# for the voronoi algorithm. 


# Visualize the output


In [ ]:
# This algo is implemented in the centrosome package, an open source image processing library.


# Visualize the output. Focus on boundaries between labeled 
# regions highlighted thanks to mark_boundaries from skimage.segmentation and draw contour lines 
# for nuclei thanks to contour from matplotlib.pyplot.


## Exercises 6 - Quantification of cell features

The ultimate goal of image segmentation is of course the extraction of quantitative measurements, in this case on a single-cell level. Measures of interest can be based on intensity (in different channels) or on the size and shape of the cells.

we will extract the following:

    Cell ID (so all other measurements can be traced back to the cell that was measured)
    Mean intensity of each cell, for each channel
    The cell area i.e. the number of pixels that make up the cell

In [ ]:
# Create a dictionary that contains a key-value pair for each measurement.
# The keys should be strings describing the type of measurement (e.g. 'cell_tubulin_mean') 
# and the values should be empty lists. These empty lists will be filled with the results of 
# the measurements and the dictionary will make it easy to work with this data.

# Record the measurements for each cell
# Iterate over the segmented cells (np.unique).
# Inside the loop, create a mask for the current cell and use it to extract the measurements listed above. 
# Add them to the appropriate list in the dictionary using the list.append method.
# Hint: Remember that you can get all the values within a masked area by indexing the image 
#       with the mask. For example, np.mean(image[cell_mask]) will return the mean of all the 
#       intensity values of 'image' that are masked by 'cell_mask'.

# Iterate over cell IDs

    # Mask the current cell
    
    
    # Get the measurements
    # Note: the .item() method ensures that the resulting number is converted from a numpy number object
    #       (e.g. type np.float) to a native python number object (e.g. type float). For most purposes,
    #       this is irrelevant, but for saving data in a python object as we do later on, it is useful
    #       to use native python objects only.
    
    
# Print the results and check that they make sense


# Writing output files

The final step of the pipeline shows how to write output of the pipeline to files.

Data can be saved to files in a human-readable format such as text files (e.g. to import into Excel), in a format readable for other programs such as tif-images (e.g. to view in Fiji).

## Exercises 7 - write a tab-separated file

In [ ]:
#  Write a tab-separated text file of the results dict
# The most generic way of saving numerical results is a simple text file. It can be imported into 
# pretty much any other program.

# To write normal text files, open an empty file object in write mode ('w') using the 'with'-statement.


    # Use the file_object.write(string) method to write strings to the file. First write the header of the
    # date (the result dict keys), separated by tabs ('\t'). It makes sense to first generate a complete
    # string with all the headers and then write this string to the file. Note that you will need to 
    # explicitly write 'newline' characters ('\n') at the end of the line to switch to the next line.
    

    # After writing the headers, iterate over all the cells saved and write the data to the file by
    # creating strings similar to the header string.
    
        
# After writing the data, have a look at the output file in a text editor or in a spreadsheet
# program like Excel.

## Exercises 8 - write a tab-separated file to omero as attachment

In [ ]:
# Import BlitzGateway from omero.gateway to access API
import sys
sys.path.append('../../OMERO.server/lib/python')
from omero.gateway import BlitzGateway

# The getpass() function prints a prompt then reads input from the user until they press return.
# The input is passed back as a string to the caller. For that import getpass from getpass.
from getpass import getpass

# Use raw_input function to print a prompt for username, host and port.
# Warning: For port parameter cast the string to integer.

# Fill the parameters of BlitzGateway function: Username, Password, Host and Port
# Use connect function to connect you to the omero server.
conn = BlitzGateway(
    raw_input("Username: "), getpass("OMERO Password: "),
    host=raw_input("Host: "), port=int(raw_input("Port: ")))
conn.connect()

# Fill the .. to run this part of the code
dataset_id = ..
dataset = conn.getObject("Dataset", ..)
# Specify a local file e.g. could be result of some analysis
file_to_upload = ..   # This file should already exist
namespace = "result_output"
print "\nCreating an OriginalFile and FileAnnotation"
file_ann = conn.createFileAnnfromLocalFile(
    file_to_upload, mimetype="text/plain", ns=namespace, desc=None)
print "Attaching FileAnnotation to Dataset: ", "File ID:", file_ann.getId(), \
    ",", file_ann.getFile().getName(), "Size:", file_ann.getFile().getSize()
dataset.linkAnnotation(file_ann)     # link it to dataset.